In [36]:
#test scrape
import json
import re
import requests
from bs4 import *
import time
import pandas as pd

In [81]:
mdf = pd.DataFrame(columns=['price', 'name'])

response = requests.get("https://www.makro.co.za/food/cooking/grains-rice-pasta/c/ICB?pageSize=20&q=%3Arelevance&page=0")
html = response.text
soup = BeautifulSoup(html, 'lxml')
pattern = '.\d$'
lastPage = int(re.search(pattern, soup.find('div', class_='mak-pagination-new').ul.text)[0]) - 1

for i in range(0, lastPage):
    response = requests.get(f"https://www.makro.co.za/food/cooking/grains-rice-pasta/c/ICB?pageSize=20&q=%3Arelevance&page={i}")
    html = response.text
    soup = BeautifulSoup(html, 'lxml')

    ### product and price
    result = soup.find_all('div', class_='mak-product-tiles-container__product-tile')
    price = result[0].find('p', class_='col-xs-12').text
    # print(price)
    name = result[0].find('a', class_='product-tile-inner__productTitle').span.text
    # print(name)

    priceList = [i.find('p', class_='col-xs-12').text for i in result]
    nameList = [i.find('a', class_='product-tile-inner__productTitle').span.text for i in result]

    df = pd.DataFrame({'price':priceList, 'name':nameList})
    mdf = pd.concat([df, mdf])

mdf
# product_names = [i.span.text for i in products]
# print(product_names)


,price,name
0,R 7595,Safari Braaipap (1 x 10kg)
1,R 3195,Allsome Parboiled Rice (2kg)
2,R 1995,Osmans Tajmahal Pea Dholl (1 x 1kg)
3,R 11100,Allsome Parboiled Rice (1 x 10kg)
4,R 5696,Champion Samp (1 x 5kg)
...,...,...
15,R 3295,Iwisa Maize Rice (1 x 2.5kg)
16,R 7195,First Value Samp (1 x 10kg)
17,R 37695,First Value Sugar Beans (1 x 10kg)
18,R 1465,Royco Packet Marinade Beef (1 x 40g)
